In [1]:
#Creatures at your level have a 5% chance to Dodge your attacks. 
#Each additional level the target has over the player grants them 0.5% additional chance
#to dodge. (So 6.5% chance to Dodge for creatures 3 levels above the player.)

In [18]:
# Note to self: I haven't implemented the following yet because these kind of auras are out of scope for cat-simulator

# In addition, when fighting +3 level mobs there's a flat modifier placed on your total crit chance gained from auras.
# Auras in this context are talents such as Cruelty or Axe Specialization, gear that directly gives crit % through 
# Equip: effects, buffs that increase you crit chance like Songflower Serenade or Leader of the Pack, and consumables such 
# as Elixir of the Mongoose. It does not include crit gained indirectly through Agility, neither base Agility or 
# Agility from gear.

In [13]:
import pandas as pd
base_stats_df = pd.read_csv('base_stats.csv', index_col = 0)
base_stats_df['Melee Crit'] = round(base_stats_df['Melee Crit'] / 100,4)
base_stats_df.head(20)

,Agility,Intellect,Strength,Stamina,Spirit,Attack Power,Melee Crit
Level,,,,,,,
1,25,20,17,19,20,34,0.0125
2,26,21,18,20,21,35,0.0193
3,26,21,18,20,21,35,0.0261
4,27,22,19,21,22,36,0.0328
5,28,23,20,22,23,37,0.0396
6,28,23,20,22,23,37,0.0464
7,29,24,21,23,24,38,0.0532
8,30,25,22,24,25,39,0.0600
9,30,25,22,24,25,39,0.0667


In [3]:
mob_armor_health_df = pd.read_html('armor_health.html',
                                   skiprows = 6,
                                   index_col = 1)
mob_armor_health_df = mob_armor_health_df[0]

cols = ['Warrior','Paladin', 'Mage', 'Warrior.1', 'Paladin.1', 'Mage.1']
mob_armor_health_df = mob_armor_health_df.loc[:,cols]
mob_armor_health_df.columns = ['Warrior_Armor','Paladin_Armor', 'Mage_Armor', 'Warrior_Health', 'Paladin_Health', 'Mage_Health']
mob_armor_health_df.head()

,Warrior_Armor,Paladin_Armor,Mage_Armor,Warrior_Health,Paladin_Health,Mage_Health
Mob Level,,,,,,
1,20,NaN,NaN,42.0,NaN,NaN
2,21,NaN,NaN,55.0,NaN,NaN
3,46,NaN,NaN,71.0,69.0,67.0
4,82,NaN,NaN,86.0,83.0,81.0
5,126,105.0,NaN,102.0,98.0,NaN


In [4]:
my_var = 3 # Declare a variable on the global scope
def my_function(my_var):
     return my_var + 1

for i in range(4):
    my_var = my_function(my_var) # call the function and assign result to global variable
    print(my_var) # check result

4
5
6
7


In [83]:
current_energy = 0
current_level = 20
mob_level = 23
TargetLevel = mob_level

AttackerSkill = 5 * current_level #dit is een synoniem voor player attack rating
mob_defense_rating = 5 * mob_level


NormalMissChance = 0.05
if TargetLevel < 10:
    MissChance = NormalMissChance * (TargetLevel / 10)
elif (mob_defense_rating - AttackerSkill) <= 10:
    MissChance = 0.05 + (TargetLevel * 5 - AttackerSkill) * 0.001
elif (mob_defense_rating - AttackerSkill) >= 11:
    MissChance = 0.05 + (TargetLevel * 5 - AttackerSkill) * 0.002
print(f'MissChance = {MissChance}')

AttackerCrit = base_stats_df.at[current_level, 'Melee Crit']
if (AttackerSkill - mob_defense_rating) < 0:
    CritChance = AttackerCrit + (AttackerSkill - mob_defense_rating) * 0.002
elif (mob_defense_rating - AttackerSkill) >= 0:
    CritChance = AttackerCrit + (AttackerSkill - mob_defense_rating) * 0.004
print(f'CritChance = {CritChance}')

if (mob_level - current_level) < 3:
    ParryChance = 0.05 + 0.005 * (mob_level - current_level)
elif (mob_level - current_level) >= 3:
    ParryChance = 0.125 + 0.005 * (mob_level - current_level)
print(f'ParryChance = {ParryChance}')

# glancing gebeurt alleen op white non-crits

#dit komt overeen met melderon maar niet met magey
GlancingChance = 0.10 + (TargetLevel - current_level) * 0.1 
print(f'GlancingChance = {GlancingChance}')
# ik krijg Glancing reduction niet goed
LowEndGlancing  = abs(1 - (1.3 - 0.05*(mob_defense_rating-AttackerSkill)))
HighEndGlancing = abs(1 - (1.2 - 0.03*(mob_defense_rating-AttackerSkill)))
GlancingReduction =  (HighEndGlancing + LowEndGlancing) / 2

print(f'LowEndGlancing = {LowEndGlancing}')
print(f'HighEndGlancing = {HighEndGlancing}')



def energy_regen(): 
    global current_energy
    global second
    if second % 2 == 0:
        if current_energy + 20 >= 100:
            current_energy = max(100, current_energy)
        elif current_energy + 20 < 100:
            current_energy += 20

for second in range(20):
    energy_regen() 
    print(f' second={second} current_energy={current_energy}') 

MissChance = 0.08
CritChance = 0.11130000000000001
ParryChance = 0.14
GlancingChance = 0.4
LowEndGlancing = 0.44999999999999996
HighEndGlancing = 0.44999999999999996
 second=0 current_energy=20
 second=1 current_energy=20
 second=2 current_energy=40
 second=3 current_energy=40
 second=4 current_energy=60
 second=5 current_energy=60
 second=6 current_energy=80
 second=7 current_energy=80
 second=8 current_energy=100
 second=9 current_energy=100
 second=10 current_energy=100
 second=11 current_energy=100
 second=12 current_energy=100
 second=13 current_energy=100
 second=14 current_energy=100
 second=15 current_energy=100
 second=16 current_energy=100
 second=17 current_energy=100
 second=18 current_energy=100
 second=19 current_energy=100


In [91]:
(abs(1 - (1.3 - 0.05*(315-300)))
 +
 abs(1 - (1.2 - 0.03*(315-300)))/2)

0.575

In [89]:
(mob_defense_rating-AttackerSkill)

15

In [58]:
1.3 - 0.05*(mob_defense_rating-AttackerSkill - 5)

0.8

In [55]:
1.3 - 0.05*(mob_defense_rating-AttackerSkill)
#* (NPC’s Defense skill – YOUR Weapon skill – 5) * 3% + 10%

0.55